In [1]:
import pandas as pd
import numpy as np
from faker import Faker
import pytest
from datetime import timedelta, datetime

fake = Faker()

In [2]:
created_at = fake.date_time_between(start_date="-3y", end_date="now")
last_updated = created_at + timedelta(days=np.random.randint(1, 500))
contract_periods = pd.DataFrame({
        "contract_period_id": range(1, 4),
        "period": [12, 24, 36],
        "created_at": created_at,
        "last_updated": last_updated
    })

details_rows = []
n = 5
for i in range(n + 1):
    created_at = fake.date_time_between(start_date="-3y", end_date="now")
    last_updated = created_at + timedelta(days=np.random.randint(1, 500))
    
    details_rows.append({
        "contract_details_id": i,
        "contract_title": fake.catch_phrase(),
        "initial_price": round(np.random.uniform(10, 200), 2),
        "discount_percent": round(np.random.choice([0, 5, 10, 15, 20]), 1),
        "contract_period_id": np.random.choice(contract_periods["contract_period_id"]),
        "num_of_sims": np.random.randint(1, 5),
        "num_of_devices": np.random.randint(0, 3),
        "personal_data_id": np.random.randint(1, 50),
        "created_at": created_at,
        "last_updated": last_updated
    })

contract_details = pd.DataFrame(details_rows)

contracts_rows = []
for i in range(n + 1):
    created_at = contract_details.iloc[i]["created_at"]
    last_updated = contract_details.iloc[i]["last_updated"]
    
    contracts_rows.append({
        "contract_id": i,
        "contract_details_id": contract_details.iloc[i]["contract_details_id"],
        "contract_type_id": np.random.choice([1,2,3]),
        "created_at": created_at,
        "last_updated": last_updated
    })

contracts = pd.DataFrame(contracts_rows)


customers_contracts_rows = []
for i in range(n + 1):
    created_at = fake.date_time_between(start_date="-3y", end_date="now")
    last_updated = created_at + timedelta(days=np.random.randint(1, 500))
    customers_contracts_rows.append({
        "customer_contract_id": i,
        "customer_id": np.random.randint(1, 50),
        "contract_id": contracts.iloc[i]["contract_id"],
        "created_at": created_at,
        "last_updated": last_updated
    })

customers_contracts = pd.DataFrame(customers_contracts_rows)

In [3]:
contract_details = contract_details.merge(contract_periods,on="contract_period_id",how="left" )

contract_details = contract_details[["contract_details_id", "period"]]
contract_details

,contract_details_id,period
0,0,36
1,1,12
2,2,12
3,3,12
4,4,24
5,5,24


In [4]:
contracts = contracts.merge(contract_details,on="contract_details_id",how="left")

contracts = contracts[["contract_id","created_at", "last_updated","period"]]

contracts

,contract_id,created_at,last_updated,period
0,0,2025-03-10 03:19:49.162003,2025-12-21 03:19:49.162003,36
1,1,2024-01-06 05:53:57.861751,2024-12-31 05:53:57.861751,12
2,2,2025-05-13 14:15:01.309365,2026-03-03 14:15:01.309365,12
3,3,2023-07-23 08:31:32.975951,2024-05-12 08:31:32.975951,12
4,4,2024-10-25 15:12:15.857579,2025-07-24 15:12:15.857579,24
5,5,2024-03-04 02:39:06.115963,2024-05-11 02:39:06.115963,24


In [5]:
customers_contracts = customers_contracts.merge(contracts, on="contract_id",how="left")


customers_contracts["created_at"] = customers_contracts["created_at"].dt.date
customers_contracts["last_updated"] = customers_contracts["last_updated"].dt.date

customers_contracts["start_date"] = customers_contracts["created_at"]
customers_contracts["end_date"] =  customers_contracts.apply(
    lambda row: row["start_date"] + pd.DateOffset(months=row["period"]),
    axis=1
)



customers_contracts["is_active"] = customers_contracts["end_date"] > pd.Timestamp.now()

customers_contracts.drop("customer_contract_id", axis=1, inplace=True)


customers_contracts

KeyError: 'created_at'